# <font color = "red">TF - IDF 

TF-IDF significa "Frequência do Termo - Frequência Inversa de Documentos". 

Essa é uma técnica para quantificar uma palavra nos documentos; geralmente calculamos um peso para cada palavra, o que significa a importância da palavra no documento e no corpus. Este método é uma técnica amplamente usada em Recuperação de Informação e Mineração de Texto.

Se eu lhe der uma frase, por exemplo, "Este edifício é tão alto". É fácil para nós entender a sentença como conhecemos a semântica das palavras e da sentença. Mas como o computador entenderá essa frase? O computador pode entender qualquer dado apenas na forma de valor numérico. Portanto, por esse motivo, vetorizamos todo o texto para que o computador possa entender melhor o texto.

Ao vetorizar os documentos, podemos executar várias tarefas, como encontrar documentos relevantes, classificação, agrupamento e assim por diante. É a mesma coisa que acontece quando você realiza uma pesquisa no Google. As páginas da web são chamadas de documentos e o texto com o qual você pesquisa é chamado de consulta. o Google mantém uma representação fixa para todos os documentos. Quando você pesquisa com uma consulta, o Google encontra a relevância da consulta com todos os documentos, classifica-os na ordem em que é relevante e mostra os principais documentos. Todo esse processo é feito usando a forma vetorizada da consulta e dos documentos. Embora os algoritmos do Google sejam altamente sofisticados e otimizados, essa é a estrutura usada.

Vamos extrair um livro inteiro, construir nossas funções para TF e IDF e então identificar as palavras mais relevantes em algumas frases do livro.

In [1]:
# Imports

import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

## Preparando os Dados

https://www.gutenberg.org/files/158/158-h/158-h.htm

In [3]:
# Carregando os dados
dados_livro_emma = nltk.corpus.gutenberg.sents('austen-emma.txt')

In [4]:
# Listas para receber as frases e as palavras do texto
dados_livro_emma_frases = []
dados_livro_emma_palavras = []

In [5]:
# Loop para tokenização

for sentences in dados_livro_emma:
 dados_livro_emma_frases.append([word.lower() for word in sentences if word.isalpha()])
 for word in sentences:
  if word.isalpha():
   dados_livro_emma_palavras.append(word.lower())

In [8]:
# Convertendo a lista de palavras em um conjunto (set)
dados_livro_emma_palavras = set(dados_livro_emma_palavras)

In [10]:
# Visualizando as frases
dados_livro_emma_frases

[['emma', 'by', 'jane', 'austen'],
 ['volume', 'i'],
 ['chapter', 'i'],
 ['emma',
  'woodhouse',
  'handsome',
  'clever',
  'and',
  'rich',
  'with',
  'a',
  'comfortable',
  'home',
  'and',
  'happy',
  'disposition',
  'seemed',
  'to',
  'unite',
  'some',
  'of',
  'the',
  'best',
  'blessings',
  'of',
  'existence',
  'and',
  'had',
  'lived',
  'nearly',
  'twenty',
  'one',
  'years',
  'in',
  'the',
  'world',
  'with',
  'very',
  'little',
  'to',
  'distress',
  'or',
  'vex',
  'her'],
 ['she',
  'was',
  'the',
  'youngest',
  'of',
  'the',
  'two',
  'daughters',
  'of',
  'a',
  'most',
  'affectionate',
  'indulgent',
  'father',
  'and',
  'had',
  'in',
  'consequence',
  'of',
  'her',
  'sister',
  's',
  'marriage',
  'been',
  'mistress',
  'of',
  'his',
  'house',
  'from',
  'a',
  'very',
  'early',
  'period'],
 ['her',
  'mother',
  'had',
  'died',
  'too',
  'long',
  'ago',
  'for',
  'her',
  'to',
  'have',
  'more',
  'than',
  'an',
  'indist

In [11]:
# Visualizando as palavras
dados_livro_emma_palavras

{'felt',
 'penetration',
 'sensation',
 'resulting',
 'rain',
 'pretending',
 'reassembled',
 'one',
 'fetching',
 'extenuations',
 'contributed',
 'off',
 'solidity',
 'reappearance',
 'brother',
 'currants',
 'recall',
 'arm',
 'apply',
 'apple',
 'happening',
 'definition',
 'stooping',
 'predicament',
 'corresponding',
 'yesterday',
 'background',
 'night',
 'affectation',
 'plotting',
 'disgraced',
 'shifted',
 'eltons',
 'nieces',
 'write',
 'conduce',
 'admiration',
 'wrist',
 'capacity',
 'applied',
 'consumption',
 'diffuse',
 'uncouthness',
 'man',
 'readiest',
 'seen',
 'basin',
 'interests',
 'indifference',
 'dubious',
 'disappointment',
 'material',
 'forward',
 'retreated',
 'completest',
 'bows',
 'long',
 'chat',
 'advisable',
 'regularity',
 'laurels',
 'misconceptions',
 'trusted',
 'both',
 'anything',
 'solemn',
 'curiosity',
 'qualify',
 'distinguish',
 'performer',
 'occupy',
 'which',
 'overcareful',
 'concern',
 'apprehensive',
 'wedding',
 'feminine',
 'merest

## Frequência do Termo

A Frequência do Termo mede a frequência de uma palavra em um documento. 

Isso depende muito do tamanho do documento e da generalidade da palavra, por exemplo, uma palavra muito comum como "era" pode aparecer várias vezes em um documento, mas se pegarmos dois documentos, um com 100 palavras e outro com 10.000, há uma alta probabilidade de que uma palavra comum como "era" possa estar mais presente no documento de 10.000 palavras. Mas não podemos dizer que o documento mais longo é mais importante que o documento mais curto. Por esse exato motivo, realizamos uma normalização no valor da frequência, dividindo a frequência com o número total de palavras no documento.

Lembre-se de que precisamos finalmente vetorizar o documento. Quando estamos planejando vetorizá-lo, não podemos considerar apenas as palavras que estão presentes nesse documento específico. Se fizermos isso, o comprimento do vetor será diferente para os dois documentos e não será possível calcular a semelhança. Então, o que fazemos é que vetorizar os documentos no vocabulário, que é a lista de todas as palavras possíveis no corpus.

Quando estamos vetorizando os documentos, verificamos a contagem de cada palavra. Na pior das hipóteses, se o termo não existir no documento, esse valor de TF específico será 0 e, em outro caso extremo, se todas as palavras no documento forem iguais, será 1. O valor final do documento normalizado estará no intervalo de [0 a 1], sendo 0, 1 inclusive.

**tf(t,d) = contagem de t em d / número de palavras em d**

Se já calculamos o valor do TF e se isso produz uma forma vetorizada do documento, por que não usar apenas o TF para encontrar a relevância entre os documentos? Por que precisamos da IDF?

Embora tenhamos calculado o valor do TF, ainda existem alguns problemas, por exemplo, palavras mais comuns como "é" terão valores muito altos, dando a essas palavras uma importância muito alta. Mas usar essas palavras para calcular a relevância produz maus resultados. 

Esse tipo de palavra comum é chamado de palavras de parada (stop words) e, embora removamos as stop words posteriormente na etapa de pré-processamento, descobrir a importância da palavra em todos os documentos e normalizando  esse valor, representa muito melhor os documentos.

In [12]:
# Função para calcular o Termo de Frequencia

def TermFreq(documento, palavra):
 doc_length = len(documento)
 ocorrencias = len([w for w in documento if w == palavra])
 return ocorrencias / doc_length

In [13]:
dados_livro_emma_frases[5]

['her',
 'mother',
 'had',
 'died',
 'too',
 'long',
 'ago',
 'for',
 'her',
 'to',
 'have',
 'more',
 'than',
 'an',
 'indistinct',
 'remembrance',
 'of',
 'her',
 'caresses',
 'and',
 'her',
 'place',
 'had',
 'been',
 'supplied',
 'by',
 'an',
 'excellent',
 'woman',
 'as',
 'governess',
 'who',
 'had',
 'fallen',
 'little',
 'short',
 'of',
 'a',
 'mother',
 'in',
 'affection']

In [14]:
len(dados_livro_emma_frases[5])

41

In [15]:
# Aplicando a Função 
TermFreq(dados_livro_emma_frases[5], "mother")

0.04878048780487805

In [16]:
TermFreq(dados_livro_emma_frases[5], "place")

0.024390243902439025

In [19]:
TermFreq(dados_livro_emma_frases[5], "her")

0.0975609756097561

Podemos então criar um dicionário (vocabulário).

Cada palavra única será identificada de forma única no objeto de dicionário. Isso é necessário para criar representações de textos. O corpus Bag of Words é criado e será necessário para a construção do modelo TF-IDF. 

O dicionário é criado pela lista de palavras. As frases/documentos, etc., podem ser convertidos em uma lista de palavras e depois alimentados nos corpora como parâmetro.

In [20]:
# Criando um Corpus Bag of Words - contabilizar todas as ocorrências

def cria_dict():
 output = {}

 for word in dados_livro_emma_palavras:
  output[word] = 0
  for doc in dados_livro_emma_frases:
   if word in doc:
    output[word] += 1
 
 return output

In [21]:
# Criando o dicionário
df_dict = cria_dict()

In [22]:
df_dict

{'felt': 111,
 'penetration': 5,
 'sensation': 6,
 'resulting': 1,
 'rain': 20,
 'pretending': 2,
 'reassembled': 1,
 'one': 404,
 'fetching': 2,
 'extenuations': 1,
 'contributed': 1,
 'off': 97,
 'solidity': 1,
 'reappearance': 1,
 'brother': 54,
 'currants': 1,
 'recall': 3,
 'arm': 13,
 'apply': 2,
 'apple': 8,
 'happening': 4,
 'definition': 2,
 'stooping': 3,
 'predicament': 1,
 'corresponding': 1,
 'yesterday': 29,
 'background': 1,
 'night': 41,
 'affectation': 2,
 'plotting': 1,
 'disgraced': 3,
 'shifted': 1,
 'eltons': 20,
 'nieces': 2,
 'write': 23,
 'conduce': 2,
 'admiration': 23,
 'wrist': 1,
 'capacity': 1,
 'applied': 3,
 'consumption': 2,
 'diffuse': 1,
 'uncouthness': 1,
 'man': 221,
 'readiest': 1,
 'seen': 72,
 'basin': 4,
 'interests': 2,
 'indifference': 14,
 'dubious': 1,
 'disappointment': 18,
 'material': 12,
 'forward': 35,
 'retreated': 2,
 'completest': 1,
 'bows': 1,
 'long': 139,
 'chat': 5,
 'advisable': 2,
 'regularity': 1,
 'laurels': 1,
 'misconceptio

In [25]:
df_dict['mother']

68

## Frquência Inversa

Para compreender o que é IDF, primeiro temos que compreender o que é DF.

A **Frequência do Documento** (DF) mede a importância do documento em todo o corpus e é muito semelhante ao TF. A única diferença é que TF é contador de frequência para um termo t no documento d, onde DF é a contagem de ocorrências do termo t no conjunto de documentos N. 

Em outras palavras, DF é o número de documentos em que a palavra está presente. Consideramos uma ocorrência se o termo consistir no documento pelo menos uma vez, não precisamos saber o número de vezes que o termo está presente.

df (t) = ocorrência de t nos documentos

Para manter isso também em um intervalo, normalizamos dividindo com o número total de documentos. Nosso principal objetivo é conhecer a informatividade de um termo, e DF é o inverso exato dele. É por isso que invertemos o DF.

**Frequência Inversa de Documentos**

IDF é o inverso da frequência do documento que mede a informatividade do termo t. Quando calcularmos o IDF, será muito baixo para as palavras que mais ocorrem, como stop words (porque stop words como "é" estão presentes em quase todos os documentos e N / df atribuirá um valor muito baixo a essa palavra). Isso finalmente resulta o que queremos, uma ponderação relativa.

idf (t) = N / df

Agora, existem alguns outros problemas com o IDF, no caso de um corpus grande, digamos 10.000, o valor do IDF explode. 

Então, para diminuir o efeito, aplicamos o log ao IDF.

Durante o tempo de consulta, quando uma palavra que não está no vocabulário ocorre, o df será 0. Como não podemos dividir por 0, suavizamos o valor adicionando 1 ao denominador.

idf (t) = log (N / (df + 1))

Finalmente, considerando um valor multiplicativo de TF e IDF, obtemos a pontuação TF-IDF, existem muitas variações diferentes de TF-IDF, mas por enquanto vamos nos concentrar nessa versão básica.

**tf-idf (t, d) = tf (t, d) * log (N / (df + 1))**

In [27]:
# Função para calcular a frequeência Inversa de Documentos

def InverseDocumentFrequency(word):
 N = len(dados_livro_emma_frases)
 try:
  df = df_dict[word] + 1
 except:
  df = 1
 return np.log(N/df)

In [28]:
# Aolica a Função 
InverseDocumentFrequency("mother")

4.717074461860344

# <font color="red">TF / IDF

In [30]:
# Função TF-IDF
def TFIDF(doc, word):
 tf = TermFreq(doc, word)
 idf = InverseDocumentFrequency(word)

 return tf * idf

In [31]:
dados_livro_emma_frases[5]

['her',
 'mother',
 'had',
 'died',
 'too',
 'long',
 'ago',
 'for',
 'her',
 'to',
 'have',
 'more',
 'than',
 'an',
 'indistinct',
 'remembrance',
 'of',
 'her',
 'caresses',
 'and',
 'her',
 'place',
 'had',
 'been',
 'supplied',
 'by',
 'an',
 'excellent',
 'woman',
 'as',
 'governess',
 'who',
 'had',
 'fallen',
 'little',
 'short',
 'of',
 'a',
 'mother',
 'in',
 'affection']

In [36]:
# Print
print(f"Mother - {str(TFIDF(dados_livro_emma_frases[5], 'mother'))}")

Mother - 0.2301011932614802


In [37]:
dados_livro_emma_frases[30]

['she',
 'had',
 'many',
 'acquaintance',
 'in',
 'the',
 'place',
 'for',
 'her',
 'father',
 'was',
 'universally',
 'civil',
 'but',
 'not',
 'one',
 'among',
 'them',
 'who',
 'could',
 'be',
 'accepted',
 'in',
 'lieu',
 'of',
 'miss',
 'taylor',
 'for',
 'even',
 'half',
 'a',
 'day']

In [38]:
print(f"Mother - {str(TFIDF(dados_livro_emma_frases[30], 'mother'))}")

Mother - 0.0
